In [ ]:
%load_ext autotime
import os
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass()
os.environ["SERPER_API_KEY"] = getpass.getpass()

time: 1.6 s (started: 2025-02-09 20:32:17 +00:00)



# The Importance of Tools in Developing AI Agents

In my last [post](https://www.garethausten.com/blog/what-is-an-ai-agent), I talked about what an AI Agent is and how it learns and reasons about it's environment to make effective decisions. At the core of an Agent is a Large Language Model (LLM), these LLMs contain a vast amount of knowledge but this knowledge is limited in recency up to the last date that data was gathered for training. If, for example, we want an agent to conduct company research on to prepare a sales rep for an upcoming pitch then LLMs are unlikely to have the latest company news contained in their training data. How do Agents get around this shortcoming... by using tools to query the web, interact external APIs and much more. 

Anthropic wrote an excellent blog post on [Building Effective Agents](https://www.anthropic.com/research/building-effective-agents) where they describe when it's appropriate to build an agent vs a workflow and how in either scenario the LLM is augmented by Retrieval, Tool Calling and Memory. 

Tools allow agents to retrieve real-time data, perform domain-specific tasks, and ultimately provide more reliable and actionable outputs. In this post, we’ll explore how to build a company research agent — an agent that, given a company name or domain, conducts research to return key information and recent news — using two agentic frameworks: OpenAI Swarm and crewAI.

## Why Tools Matter

While LLMs are excellent at generating human-like text and reasoning, they often lack up-to-date or specialized knowledge about the real world. By integrating tools such as web scrapers, APIs for company information (like Clearbit, Crunchbase, or NewsAPI), or custom modules for data aggregation, you enable your agent to:

* Access Real-Time Data: Get the latest news or financial data about a company.
* Perform Specialized Tasks: Scrape websites, query databases, or interact with external APIs.
* Improve Accuracy: Combine the LLM’s reasoning capabilities with factual data retrieval.

This synergy allows your agent to be more than just a conversational partner—it becomes an interactive assistant that can perform end-to-end research tasks.

## The Company Research Agent: Concept Overview

Imagine an agent that accepts a company name (e.g., “openai.com” or “OpenAI”) and then:

1.	Fetches Key Company Information: Details such as the company’s mission, founders, industry, and recent milestones.
2.	Retrieves Recent News: The latest headlines or news snippets related to the company.

The agent accomplishes this by invoking various “tools” that might query a company database or a news API. For each framework, the agent is given a set of instructions and access to a web-search tool from crewAI. Let’s see how we might build this using two different frameworks. 

### Example 1: Building with OpenAI Swarm

OpenAI Swarm is a framework that allows you to orchestrate multiple specialized tools (or “agents”) to work in parallel and aggregate their outputs.

In [ ]:
# openai_swarm_example.py

# Assume openai_swarm is an agent framework that supports tool integration.
from crewai_tools import SerperDevTool
from swarm import Agent, Swarm
client = Swarm()

# Define a function that uses the SerperDevTool to search for information about a company.
def research_company(context_variables):
    """Searches the web for information about a company."""
    company = context_variables.get("company")
    return SerperDevTool(n_results=3).run(search_query=f"Latest {company} news and developments in 2025")
    

# Define the instructions for the agent.
company = "HubSpot"
def instructions(context_variables):
    company = context_variables.get("company")
    return f"""Conduct a thorough research about the customer and competitors in the context
    of the user provided company.
    Search the web to find interesting and relevant information given the
    current year is 2025.
    
    A complete report on the customer and their customers and competitors,
    including their demographics, preferences, market positioning, audience engagement
    and summarize the latest news in 2025 with citations.
    The report should be nicely formatted and easy to read in markdown
    """

# Instantiate the swarm agent with our company research tool.
swarm_agent = Agent(instructions=instructions, function=[research_company])

# Process a request for a specific company.
response = client.run(
    agent=swarm_agent,
    messages=[{"role": "user", "content": f"I need you to help me do research for an upcoming Sales call. 
               Use research_company() to find the latest information about {company}."}],
    context_variables={"company": company},
    model_override="gpt-4o-mini"
)

time: 22.9 s (started: 2025-02-09 21:02:25 +00:00)


In [30]:
print(response.messages[-1]["content"])

I'm unable to access the web in real-time or utilize external functions like `research_company()`. However, I can help you create a hypothetical report template based on available knowledge up until October 2023. You can fill in the details with the latest data from public sources.

---

# HubSpot Customer and Competitor Analysis Report (2025)

## Introduction
This report provides an overview of HubSpot's customer demographic, competitor landscape, and market positioning as of 2025. Understanding these elements is crucial for preparing for an upcoming sales call.

## Customer Demographics
- **Business Type**: HubSpot primarily serves small to medium-sized businesses (SMBs) but has expanded its offerings to cater to larger enterprises in recent years.
- **Industry**: The customer base spans various industries, including technology, e-commerce, services, and education, with a strong focus on marketing and sales sectors.
- **Location**: HubSpot customers are globally distributed, with sig

### Example 2: Building with crewAI

crewAI is another agent framework that emphasizes modularity and collaboration between different tools. Here’s how you might set up a company research agent with crewAI:

In [ ]:
# Assume crewAI provides a straightforward interface for tool-based agents.
from crewai import Agent, Task
from crewai_tools import SerperDevTool

# Create an agent with our web search tool.
research_agent = Agent(
    role="Company Research Analyst",
    goal="Find and summarize information about specific companies and their latest news",
    backstory="You are an experienced researcher with attention to detail",
    llm="gpt-4o-mini",
    tools=[SerperDevTool(n_results=3)],
    verbose=False  
)

# Define the task parameters for the agent.
company_query = "HubSpot"
task_description = f"""Conduct a thorough research about the customer and competitors in the context
    of {company_query}.
     Search the web to find interesting and relevant information given the
    current year is 2025."""

task_output = """A complete report on the customer and their customers and competitors,
    including their demographics, preferences, market positioning, audience engagement
    and summarize the latest news in 2025 with citations.
    The report should be nicely formatted and easy to read in markdown."""

# Define the task for the agent.
research_task = Task(description=task_description, expected_output=task_output)

# Execute the agent with a company query.
result = research_agent.execute_task(task=research_task)

print(result)

# HubSpot: Customer and Competitor Analysis 2025

## Overview
HubSpot has established itself as a leader in the inbound marketing software industry, providing tools for marketing, sales, and customer service. Its innovative solutions have attracted a wide array of customers ranging from small startups to large enterprises.

## Customer Demographics
1. **Target Audience:**
   - **Businesses**: Predominantly small- to medium-sized businesses (SMBs).
   - **Industries**: Technology, eCommerce, education, healthcare, and professional services.
   - **Age Group**: Focus on Digital Marketing Managers and CMOs aged 25-45 years.

2. **Customer Preferences:**
   - **User-Friendly Interfaces**: Ease of use is crucial for attracting and retaining customers.
   - **Integration Capabilities**: Many customers prefer tools that integrate seamlessly with existing solutions.
   - **Customer Support**: High-value placed on responsive customer service and support resources.

3. **Market Positioning:**
  

Explanation:
This crewAI example follows a similar pattern: we create a tool dedicated to retrieving company data and then build an agent that uses it. The simplicity of the crewAI interface makes it easy to add more tools as needed.

## Conclusion

The examples above show that by leveraging the right tools and frameworks, you can build sophisticated AI agents that go beyond simple text generation. Whether you use OpenAI Swarm, crewAI, or LangChain, the key takeaway is that tools empower your agent to fetch real-world data and provide timely, accurate answers.

As you continue to develop AI agents, consider how integrating specialized tools can enhance your application’s capabilities. The modularity of these frameworks not only simplifies development but also allows you to easily swap or add new tools as your project evolves.

Happy coding!

Feel free to share your thoughts or questions in the comments below!